# Tensorflow versus Poisonous Mushrooms

In [10]:
from pandas import read_csv
srooms_df = read_csv('../data/agaricus-lepiota.data.csv')
from sklearn_pandas import DataFrameMapper
import sklearn
import numpy as np

mappings = ([
    ('edibility', sklearn.preprocessing.LabelEncoder()),
    ('odor', sklearn.preprocessing.LabelBinarizer()),
    ('habitat', sklearn.preprocessing.LabelBinarizer()),
    ('spore-print-color', sklearn.preprocessing.LabelBinarizer())
])

mapper = DataFrameMapper(mappings)
srooms_np = mapper.fit_transform(srooms_df.copy()).astype(np.float32)

from sklearn.model_selection import train_test_split
train, test = train_test_split(srooms_np, test_size = 0.2, random_state=7)
train_labels = train[:,0:1]
train_data = train[:,1:]
test_labels = test[:,0:1]
test_data = test[:,1:]

In [22]:
import tensorflow as tf
import math
def inference(samples, input_dim, dense1_units, dense2_units):
    with tf.name_scope('dense_1'):
        weights = tf.Variable(
            tf.truncated_normal([input_dim, dense1_units],
                                stddev=1.0 / math.sqrt(float(input_dim))),
            name='weights')
        biases = tf.Variable(tf.zeros([dense1_units]),
                             name='biases')
        dense1 = tf.nn.relu(tf.nn.xw_plus_b(samples, weights, biases))
        
    with tf.name_scope('dropout'):
        dropout = tf.nn.dropout(dense1, 0.5)
        
    with tf.name_scope('dense_2'):
        weights = tf.Variable(
            tf.truncated_normal([dense1_units, dense2_units],
                                stddev=1.0 / math.sqrt(float(dense2_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([dense2_units]),
                             name='biases')
        logits = tf.sigmoid(tf.nn.xw_plus_b(dropout, weights, biases))
        return logits

In [31]:
def loss(logits, labels):
  xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
  return tf.reduce_mean(xentropy)

def predict(logits):
    return tf.round(logits)

def accuracy(logits, labels):
    return tf.reduce_mean(tf.to_float(tf.equal(predict(logits),labels)))

def training(loss):
    tf.summary.scalar('loss', loss)
    optimizer = tf.train.AdamOptimizer()
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op

In [35]:
x = tf.placeholder(tf.float32, shape=train_data.shape)
y = tf.placeholder(tf.float32, shape=train_labels.shape)

logits = inference(x, 25, 20, 1)
loss_op = loss(logits, y)
predict_op = predict(logits)
acc_op = accuracy(logits, y)
train_op = training(loss_op)
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    epochs = 1000
    for epoch in range(epochs):
        _, loss_val, acc_val = sess.run([train_op, loss_op, acc_op], feed_dict={x: train_data, y: train_labels})
        if epoch % 100 == 0:
            print('loss: {}, accuracy: {}'.format(loss_val,acc_val))
    sess.close()

loss: 0.7149622440338135, accuracy: 0.520080029964447
loss: 0.6092740297317505, accuracy: 0.8598245978355408
loss: 0.5564351081848145, accuracy: 0.9478381276130676
loss: 0.5363012552261353, accuracy: 0.9736882448196411
loss: 0.5266803503036499, accuracy: 0.9856901168823242
loss: 0.5221099257469177, accuracy: 0.9892290830612183
loss: 0.5196435451507568, accuracy: 0.991844892501831
loss: 0.5184599757194519, accuracy: 0.9924603700637817
loss: 0.5171313285827637, accuracy: 0.9927681088447571
loss: 0.5161339044570923, accuracy: 0.9933835864067078
